In [ ]:
!pip install gspread google-auth flask

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken your-token-here

In [ ]:
from pyngrok import ngrok

# Open a tunnel to the Flask app
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


In [ ]:
import gspread
from google.oauth2.service_account import Credentials
from datetime import datetime
from flask import Flask, request, jsonify

app = Flask(__name__)

# Google Sheets authentication
scopes = ["https://www.googleapis.com/auth/spreadsheets"]
creds = Credentials.from_service_account_file("credentials.json", scopes=scopes)
client = gspread.authorize(creds)

@app.route('/update-attendance', methods=['POST'])
def update_attendance():
    # Connect to the Google Sheet
    data = request.get_json()
    sheet_id = data.get('sheet_id')
    print(f"Received Sheet ID: {sheet_id}")
    workbook = client.open_by_key(sheet_id)
    sheet = workbook.sheet1  # assuming your data is on the first sheet

    # Retrieve sheet data
    date_columns = [i for i, col in enumerate(sheet.row_values(1)) if "/" in col]
    latest_date_col = date_columns[-1] + 1  # latest date column index in 1-based

    # Color scheme
    color = {
        "P": {"backgroundColor": {"red": 0.0, "green": 1.0, "blue": 0.0, "alpha": 0.2}},  # Green for Present
        "A": {"backgroundColor": {"red": 1.0, "green": 0.0, "blue": 0.0, "alpha": 0.2}},  # Red for Absent
        "defaulter": {"backgroundColor": {"red": 1.0, "green": 1.0, "blue": 0.0, "alpha": 0.2}}  # Yellow for Defaulters
    }

    # First, mark attendance
    for i, row in enumerate(sheet.get_all_records(), start=2):  # start=2 to skip header row
        in_time = row['in-time']
        out_time = row['out-time']

        # Check attendance if out_time is missing
        if not out_time:
            presence = "A"
        else:
            if in_time and out_time:
                time_format = "%H:%M:%S"
                in_time_obj = datetime.strptime(in_time, time_format)
                out_time_obj = datetime.strptime(out_time, time_format)
                duration = (out_time_obj - in_time_obj).total_seconds() / 60  # duration in minutes

                # Set threshold for present (e.g., 5 minutes)
                presence = "P" if duration >= 5 else "A"

        # Update cell for the latest date column with "P" or "A"
        sheet.update_cell(i, latest_date_col, presence)

        # Apply color formatting
        cell_range = f"{chr(64 + latest_date_col)}{i}"
        sheet.format(cell_range, color[presence])

    # Retrieve updated data
    updated_data = sheet.get_all_records()

    # Calculate and update attendance percentage
    for i, row in enumerate(updated_data, start=2):
        total_classes = sum(1 for j in date_columns if row[sheet.row_values(1)[j]] in ["P", "A"])
        attended_classes = sum(1 for j in date_columns if row[sheet.row_values(1)[j]] == "P")
        attendance_percentage = (attended_classes / total_classes) * 100 if total_classes > 0 else 0
        sheet.update_cell(i, sheet.find("percentage").col, f"{attendance_percentage:.2f}%")

        # Check if attendance percentage is less than 50% and mark as defaulter
        if attendance_percentage <= 50:
            # Highlight the entire row (up to the percentage column)
            row_range = f"A{i}:{chr(64 + sheet.find('percentage').col)}{i}"
            sheet.format(row_range, color["defaulter"])

    return jsonify({"status": "success"}), 200


In [ ]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)